In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import time
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
import os
import glob
import pickle
from moviepy.editor import *
# assigning a random seed so that the results are reproducible
np.random.seed(42)
%matplotlib inline

In [2]:
# images are divided into vehicles and non-vehicles folders and each contain more subfolders
# first extract all cars images
basedir = 'vehicles/'
image_types = os.listdir(basedir)
cars= []
for imtype in image_types:
    cars.extend(glob.glob(basedir+imtype+'/*'))
    
print('Number of vehicle images found: ', len(cars))

# extract all non-car images
basedir = 'non-vehicles/'
image_types = os.listdir(basedir)
notcars= []
for imtype in image_types:
    notcars.extend(glob.glob(basedir+imtype+'/*'))
    
print('Number of non-vehicle images found: ', len(notcars))
# write these files 
with open("cars.txt", "w") as f:
    for fn in cars:
        f.write(fn+'\n')
with open("noncars.txt", "w") as f:
    for fn in notcars:
        f.write(fn+'\n')

Number of vehicle images found:  8792
Number of non-vehicle images found:  8968


In [17]:
# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=False, visualise=True, feature_vector=feature_vec)
        return features, hog_image
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block),
                       transform_sqrt=False, visualise=False, feature_vector=feature_vec)
        return features

# Define a function to compute binned color features
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    color1 = cv2.resize(img[:, :, 0], size).ravel()
    color2 = cv2.resize(img[:, :, 1], size).ravel()
    color3 = cv2.resize(img[:, :, 2], size).ravel()
    # Return the feature vector
    return np.hstack((color1, color2, color3))

# Define a function to compute color histogram features  
def color_hist(img, nbins=32):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

In [10]:
# Define a function to extract features from a list of images
# Have this function call bin_spatial(), color_hist() and hog
def extract_features(imgs, color_space='YCrCb', orient=9, pix_per_cell=8, cell_per_block=2, 
                     hog_channel=0, spatial_size=(32, 32),
                     hist_bins=32, spatial_feat=True, hist_feat=True, hog_feat=True):
    
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        image_features = []
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if color_space != 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)

        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            image_features.append(spatial_features)
        
        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            image_features.append(hist_features)
        
        if hog_feat == True:
        # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel],
                                        orient, pix_per_cell, cell_per_block,
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient,
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            
            # Append the new feature vector to the features list
            image_features.append(hog_features)
        features.append(np.concatenate(image_features))
        
    # Return list of feature vectors
    return features

In [11]:
# train the Linear Support Vector Classifier
def train(cars, notcars):
    color_space = 'YCrCb'
    orient = 9
    pix_per_cell = 8
    cell_per_block = 2
    hog_channel = 'ALL'
    spatial_size = (32, 32)
    hist_bins = 32
    spatial_feat = True
    hist_feat = True
    hog_feat = True
    car_features = extract_features(cars, color_space, orient, pix_per_cell, cell_per_block, 
                                    hog_channel, spatial_size, hist_bins, spatial_feat, 
                                    hist_feat, hog_feat)
    non_car_features = extract_features(notcars, color_space, orient, pix_per_cell, 
                                        cell_per_block, hog_channel, spatial_size,
                                        hist_bins, spatial_feat, hist_feat, hog_feat)
    print("features extracted")
    
    # in an image we are mostly going to find a lot of non-cars elements and
    # few of cars. If we use balanced data, our data might start predicting
    # more cars than needed. So, we need to 'balance' our dataset in a way
    # that the classifier sees more non-car samples than car samples but not
    # so less that it cannot learn the cars at all. One way of doing this in
    # machine learning is to oversample one class that is expected more to be
    #  seen in real scenario. We can double the sample space of non-cars dataset
    X = np.vstack((car_features,
                   non_car_features,
                   non_car_features
                  )).astype(np.float64)
    print(X.shape)
    # assigning labels to both classes where cars = 1 and non-cars = 0
    # and also accommodating for the fact that the non-car elements are
    # oversampled
    y = np.hstack((np.ones(len(car_features)),
                   np.zeros(2*len(non_car_features))))
    print("Shape of data: ", X.shape)
    print("Shape of labels: ", y.shape)
    # normalizing the training set
    scalar = StandardScaler()
    # fitting the scalar to training data
    scalar.fit(X)
    # transforming the data according to the scalar
    normalized_X = scalar.transform(X)

    # split the dataset into train and test
    X_train, X_test, y_train, y_test = train_test_split(normalized_X, y, test_size=0.2)
    print("Shape of training data: ", X_train.shape)
    print("Shape of train labels: ", y_train.shape)
    print("Shape of test data: ", X_test.shape)
    print("Shape of test labels: ", y_test.shape)
    clf = LinearSVC()
    # fit the classifier to the training data
    # Check the training time for the SVC
    t = time.time()
    clf.fit(X_train, y_train)
    t2 = time.time()
    print(round(t2 - t, 2), 'Seconds to train SVC...')
    # Check the score of the SVC
    print('Test Accuracy of SVC = ', round(clf.score(X_test, y_test), 4))

In [12]:
# training model on YCrCb
train(cars, notcars)

features extracted
(26728, 8460)
Shape of data:  (26728, 8460)
Shape of labels:  (26728,)
Shape of training data:  (21382, 8460)
Shape of train labels:  (21382,)
Shape of test data:  (5346, 8460)
Shape of test labels:  (5346,)
45.88 Seconds to train SVC...
Test Accuracy of SVC =  0.9961


In [13]:
def final_model(cars, notcars):
    color_space = 'YCrCb'
    orient = 9
    pix_per_cell = 8
    cell_per_block = 2
    hog_channel = 'ALL'
    spatial_size = (32, 32)
    hist_bins = 32
    spatial_feat = True
    hist_feat = True
    hog_feat = True
    car_features = extract_features(cars, color_space, orient, pix_per_cell, cell_per_block, 
                                    hog_channel, spatial_size, hist_bins, spatial_feat, 
                                    hist_feat, hog_feat)
    non_car_features = extract_features(notcars, color_space, orient, pix_per_cell, 
                                        cell_per_block, hog_channel, spatial_size,
                                        hist_bins, spatial_feat, hist_feat, hog_feat)
    print("features extracted")
    # in an image we are mostly going to find a lot of non-cars elements and
    # few of cars. If we use balanced data, our data might start predicting
    # more cars than needed. So, we need to 'balance' our dataset in a way
    # that the classifier sees more non-car samples than car samples but not
    # so less that it cannot learn the cars at all. One way of doing this in
    # machine learning is to oversample one class that is expected more to be
    #  seen in real scenario. We can double the sample space of non-cars dataset
    X = np.vstack((car_features,
                   non_car_features,
                  non_car_features)).astype(np.float64)
    # assigning labels to both classes where cars = 1 and non-cars = 0
    # and also accommodating for the fact that the non-car elements are
    # oversampled
    y = np.hstack((np.ones(len(car_features)),
                   np.zeros(2*len(non_car_features))))
    print("Shape of data: ", X.shape)
    print("Shape of labels: ", y.shape)
    # normalizing the training set
    scalar = StandardScaler()
    # fitting the scalar to training data
    scalar.fit(X)
    # transforming the data according to the scalar
    normalized_X = scalar.transform(X)
    final_clf = LinearSVC()
    # fitting the model on all the given
    final_clf.fit(normalized_X, y)
    
    
    # save the classifier
    with open(color_space + '_linear_svm_best.pkl', 'wb') as final_model:
        pickle.dump(final_clf, final_model)
        
    with open(color_space + "_scaling_model.pkl", "wb") as scalar_model:
        pickle.dump(scalar, scalar_model)

In [15]:
# final model on YCrCb
final_model(cars, notcars)

features extracted
Shape of data:  (26728, 8460)
Shape of labels:  (26728,)
